### Here we selected 1000 recipes that requires being labelled by the AMT workers
The details we used here is too complicated. Actually, a random sampling and strarified to nutrition property will be better. <br>
Therefore, we did not write a detailed documentation for this notebook

In [4]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2
# add path
import os 
import sys 
parent_dir = os.path.abspath(os.getcwd()+'/..')+'/' 
sys.path.append(parent_dir) 

from utils.path import dir_HugeFiles
from utils.preprocessing import load, preprocessing

from utils.save import make_dir, save_pickle, load_pickle, auto_save_csv

from sync_google.access_gsheet import save_to_google, amt

import pandas as pd
import numpy as np
import seaborn as sns

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.25 µs
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
time: 200 ms


### load the two files

In [11]:
dir_save0 = os.path.normpath(dir_HugeFiles+'dph/allrecipes.pickle')
data = load(dir_save0)
cols = [col for col in data.columns if col[0] == '_' and col not in ['_time','_followers_count']]
dic_origin = data.dropna(subset = cols).reset_index(drop=True).to_dict(orient = 'index')

dir_save = os.path.normpath(dir_HugeFiles+'dph/dic_20190607.pickle')
dic = load(dir_save)
len(dic),len(dic_origin)

exist
exist


(55102, 55102)

time: 16.6 s


### drop those with only one ingredient or direction

In [13]:
ls = [i for i,v in dic.items() if len(v['ingredients'])>1]
print('drop %d recipes with less than 2 ingredients' %(len(dic)-len(ls)))

drop 46 recipes with less than 2 ingredients
time: 120 ms


In [14]:
ls = [i for i in ls if len(dic[i]['directions'])>1]
print('furthur drop %d recipes with less than 2 instructions' %(len(dic)-len(ls)))

furthur drop 1026 recipes with less than 2 instructions
time: 182 ms


In [15]:
changeid= dict(zip(range(len(ls)), ls))

time: 70.5 ms


In [16]:
df = pd.read_csv('csv/2019-04-16 09:27:05.967004.csv')
df['origin_id'] = [changeid[v] for v in df['test_index_']]
df['sugars/dietary fiber'] = [dic[i]['sugars/dietary fiber'] for i in df['origin_id']]

time: 478 ms


In [17]:
nu = 'sugars/dietary fiber'
bins1 = [-1, 1, 13, np.inf]
names1 =['0-1','1-13','13-nan']
df[nu+'_level'] = pd.cut(df[nu], bins1, labels=names1)
df.groupby([nu+'_level']).agg(['count'])

,conditions,true0,true1,pred0,pred1,cross_entropy,true0_,true1_,pred0_,pred1_,cross_entropy_,test_index_,ranking_sum,origin_id,sugars/dietary fiber
,count,count,count,count,count,count,count,count,count,count,count,count,count,count,count
sugars/dietary fiber_level,,,,,,,,,,,,,,,
0-1,12289,12289,12289,12289,12289,12289,12289,12289,12289,12289,12289,12289,12289,12289,12289
1-13,27365,27365,27365,27365,27365,27365,27365,27365,27365,27365,27365,27365,27365,27365,27365
13-nan,14013,14013,14013,14013,14013,14013,14013,14013,14013,14013,14013,14013,14013,14013,14013


time: 138 ms


In [18]:
d = dict(zip(['0-1','1-13','13-nan'],[334,333,333])) # number_to_select
gb = df.groupby(nu+'_level').apply(lambda dfg: dfg.nlargest(d[dfg.name],'ranking_sum'))

time: 100 ms


In [19]:
df[df['origin_id']==12144]

,conditions,true0,true1,pred0,pred1,cross_entropy,true0_,true1_,pred0_,pred1_,cross_entropy_,test_index_,ranking_sum,origin_id,sugars/dietary fiber,sugars/dietary fiber_level
240,240,1.0,0.0,0.756156,0.243844,0.279508,1.0,0.0,0.129748,0.870252,2.042159,11899.0,103737.0,12144,1.290323,1-13


time: 81.3 ms


In [20]:
ids_1000 = gb.origin_id.values

time: 57.7 ms


In [21]:
ids_1000 = np.random.permutation(ids_1000)

time: 58.1 ms


In [22]:
import re
def modify_str(directions):
    def modify(text):
        text = re.sub(r'\s([()?.!:,])',r'\1',text)
        text = re.sub("([(] )","(",text)
        return text[0].upper()+text[1:]
    return [modify(text) for text in directions]

time: 68 ms


In [23]:
num, recipes = 0,[]
for i in ids_1000:
    v = dic[i]
    recipe, food = '', ''
    num+=1
    food+= 'No. %d, (RecipeID: %05d)<br>'%(num,i)
    recipe+= '%s: %s<br><br>'%('Name',v['name'])
    recipe+= '%s:<br> %s<br><br>'%('Ingredients','<br>  '.join(modify_str(v['ingredients'])))
    recipe+= '%s:<br> %s<br><br>'%('Directions','<br>  '.join(modify_str(v['directions'])))
    recipes.append([food,recipe])
df_tosave = pd.DataFrame(recipes, columns = ['id','recipes'])

time: 189 ms


In [24]:
auto_save_csv(df_tosave)

save to csv/2019-04-16 09:49:15.582033.csv


,id,recipes
0,"No. 1, (RecipeID: 53471)<br>",Name: grilled soy sesame asparagus<br><br>Ingr...
1,"No. 2, (RecipeID: 00302)<br>",Name: lemongrass and citrus poached salmon<br>...
2,"No. 3, (RecipeID: 51000)<br>",Name: lamb chops in duck sauce<br><br>Ingredie...
3,"No. 4, (RecipeID: 11299)<br>",Name: ward eight cocktail<br><br>Ingredients:<...
4,"No. 5, (RecipeID: 54674)<br>",Name: quick polynesian chicken<br><br>Ingredie...
5,"No. 6, (RecipeID: 49361)<br>",Name: shrimp and feta cheese pasta<br><br>Ingr...
6,"No. 7, (RecipeID: 11035)<br>",Name: italian chicken marinade<br><br>Ingredie...
7,"No. 8, (RecipeID: 42747)<br>",Name: russian cheese salad<br><br>Ingredients:...
8,"No. 9, (RecipeID: 37717)<br>",Name: sweet pepper pasta toss<br><br>Ingredien...
9,"No. 10, (RecipeID: 07543)<br>",Name: finadene sauce<br><br>Ingredients:<br> 1...


time: 183 ms


In [26]:
df_tosave = pd.read_csv('csv/2019-04-16 09:49:15.582033.csv')[['id','recipes']]

time: 79.1 ms


In [30]:
j, overwrite, subset = 0, True, 10
for i in range(0, 1000, subset):
    j = int(i/200)+1
    num = int(i/subset)+1
    save_to_google(form['part%d'%(j)], 'subset %2d'%(num) , df_tosave.iloc[i:i+subset].reset_index(drop=True), overwrite = False)

already exist
did not overwrite
already exist
did not overwrite
already exist
did not overwrite
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
save
time: 6min 34s


In [40]:
num, recipes = 0,[]
for i in ids_1000:
    v = dic[i]
    ids, recipe = '', ''
    ids+= 'RecipeID: %05d<br>'%(i)
    recipe+= '%s: %s<br><br>'%('Name',v['name'])
    recipe+= '%s:<br> %s<br><br>'%('Ingredients','<br>  '.join(modify_str(v['ingredients'])))
    recipe+= '%s:<br> %s<br><br>'%('Directions','<br>  '.join(modify_str(v['directions'])))
    recipes.append([ids,recipe])
df_tosave = pd.DataFrame(recipes, columns = ['id','recipes'])
auto_save_csv(df_tosave)
save_to_google(amt['1000Q'], 'version2' , df_tosave.reset_index(drop=True), overwrite = True, row_names = False)

save to csv/2019-04-03 03:15:20.534940.csv


,id,recipes
0,RecipeID: 27565<br>,Name: buckwheat tortilla pizza<br><br>Ingredie...
1,RecipeID: 52998<br>,Name: czech cabbage dish<br><br>Ingredients:<b...
2,RecipeID: 12936<br>,Name: breaded parmesan chicken<br><br>Ingredie...
3,RecipeID: 11183<br>,Name: ginger swordfish<br><br>Ingredients:<br>...
4,RecipeID: 04661<br>,Name: taiwanese pork chops<br><br>Ingredients:...
5,RecipeID: 36771<br>,Name: lamb chops with minted yogurt sauce<br><...
6,RecipeID: 26594<br>,Name: 100 percent whole wheat bread<br><br>Ing...
7,RecipeID: 31675<br>,Name: madras cocktail<br><br>Ingredients:<br> ...
8,RecipeID: 18565<br>,Name: cherry limeade ii<br><br>Ingredients:<br...
9,RecipeID: 29834<br>,Name: mean woman pasta<br><br>Ingredients:<br>...


already exist
overwrite
time: 16.7 s


In [41]:
save_to_google(amt['1000Q'], 'version2_first100' , df_tosave.iloc[:100].reset_index(drop=True), overwrite = False, row_names = False)

save
time: 7.66 s
